## Imports

In [11]:
import pandas as pd
import numpy as np
import seaborn as sns
import warnings
from scipy import stats
import pandas as pd
import plotly.express as px
import matplotlib as plt
from sklearn.impute import KNNImputer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import cross_validate
import pickle
warnings.simplefilter(action="ignore")

In [2]:
#Chargez le dataframe
data=pd.read_csv('data.csv')

In [13]:
#Import des modèles
linear_regression = pickle.load(open('linear_regression.sav', 'rb'))
random_forest = pickle.load(open('random_forest.sav', 'rb'))

## Transformations

In [ ]:
#Imputez les valures nulles
imputer = KNNImputer()

# Convert the 1D array to a 2D array with a single column
total_bedrooms = data['total_bedrooms'].values.reshape(-1, 1)

# Impute missing values using KNN imputation
imputed_bedrooms = imputer.fit_transform(total_bedrooms)

# Replace the original column with the imputed column
data['total_bedrooms'] = imputed_bedrooms

In [ ]:
#Bin des âges pour rêgler le problème d'outliers

data['binned_age']=pd.cut(data['housing_median_age'],
                        bins = [0,10,20,30,40,50,np.inf], 
                        labels = ['[0-10]','[10-20]','[20-30]','[30-40]','[40-50]','[50+]'])
data = data.drop(data[['housing_median_age']],axis=1)
data

In [ ]:
#Encodage des variables qualitatives

#creating instance of one-hot-encoder
encoder = OneHotEncoder(handle_unknown='ignore')

#perform one-hot encoding on 'team' column 
encoder_df = pd.DataFrame(encoder.fit_transform(data[['ocean_proximity','binned_age']]).toarray())

#merge one-hot encoded columns back with original DataFrame
data = data.join(encoder_df)

#Drop the now unnecessary columns
data=data.drop(['ocean_proximity','binned_age'],axis=1)

In [ ]:
columns = ['total_rooms','population','households','median_income','median_house_value','total_bedrooms']

In [ ]:
#Creation de colonne outlier pour ne pas perdre l'information avec le log
data['outlier'] = 0
for col in data[columns].columns:
    Q1 = data[col].quantile(0.25)
    Q3 = data[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    data.loc[(data[col] < lower_bound) | (data[col] > upper_bound), 'outlier'] = 1

In [ ]:
#Transformation log
data_log = np.log(abs(data[columns].values))
data[columns]=data_log

## Predictions

In [ ]:
X = data.drop(['median_house_value'],axis=1)
y=data['median_house_value']

### Linear Regression

In [ ]:
y_pred = linear_regression.predict(X)
y_pred_original_scale = np.exp(predictions)

### Random Forest

In [ ]:
y_pred = linear_regression.predict(X)
random_forest = np.exp(predictions)